<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><b>Persist, Deploy, and Score an SPSS Model to Predict Customer Churn</b></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
  <tr style="border: none">
       <th style="border: none"><img src="https://github.com/pmservice/wml-sample-models/blob/master/spark/customer-satisfaction-prediction/images/users_banner_2-03.png?raw=true" width="900" alt="Icon"> </th>
   </tr>
</table>

This notebook demonstrates how to deploy an SPSS model and score test data against the SPSS model. 

You will use the **Telco Customer Churn** data set which contains anonymous data about customers of a telecommunication company. Use the details of this data set to predict customer churn which is critical to business as it is easier to retain existing customers rather than acquire new ones.

Some familiarity with Python is helpful. This notebook is compatible with Watson Studio Desktop 1.1, Watson Machine Learning Server 1.1, and Python 3.6.


## Learning goals

In this notebook, you learn how to:

-  Set up a Watson Machine Learning Server.
-  Deploy the SPSS model online.
-  Score data against the deployed model.


## Contents

1.	[Setting up](#setup)  
    1.1 [Connecting to Watson Studio Desktop](#wsd)  
    1.2 [Connecting to Watson Machine Learning Server](#wmls)  
2.	[Save the SPSS model](#save)  
3.  [Deploy the SPSS model](#deploy)  
    3.1 [Create a model deployment](#create)  
    3.2 [Score the model](#score)  
    3.3 [Delete the deployment and model](#delete)  
4.	[Summary](#summary)

To get started on Watson Machine Learning (WML) Server, find documentation on installion and set up <a href="https://www.ibm.com/support/knowledgecenter/SS3PWM_1.0.0/wsj/wmls/wmls-install-over.html" target="_blank" rel="noopener no referrer">here</a>.

## 1. Setting up <a id="setup"></a>

In this section, you will learn how to use the python client to connect to both `Watson Studio Desktop (WSD)` and `Watson Machine Learning (WML) Server`. If you only intend to save the model on WSD, you will need to follow the steps in section [1.1 Connecting to Watson Studio Desktop](#wsd).

If you want to use the WML Server, you will need to refer to section [1.2 Connecting to Watson Machine Learning Server](#wmls). From there, you will be able to save, deploy, and score the model in your deployment space on the WML Server.

<div class="alert-block alert-info">
    <br>&nbsp;&nbsp;&nbsp;&nbsp;You can find more information about the watson-machine-learning-client <a href="https://wml-api-pyclient-dev-v4.mybluemix.net/">here</a>.
    <br><br>
</div>

First, import the `watson-machine-learning-client` module and authenticate the service instance.

<div class="alert alert-block alert-warning">
To simply hide the output of pip install, use <tt>-q</tt> after <tt>!pip install</tt>.
</div>

In [1]:
!pip install -q --upgrade watson-machine-learning-client-V4

In [2]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

### 1.1 Connecting to Watson Studio Desktop <a id="wsd"></a>

To associate the python client with Watson Studio Desktop, use the following credentials.

In [3]:
from project_lib.utils import environment

url = environment.get_common_api_url()
wml_credentials = {
    'instance_id': 'wsd_local',
    'url': url,
    'version': '1.1'
}

Now, instantiate a `WatsonMachineLearningAPIClient` object.

In [4]:
client = WatsonMachineLearningAPIClient(wml_credentials)

In [5]:
client.version

'1.0.112'

Setting the default project is mandatory when you use WSD. You can use the cell below.

In [6]:
from project_lib import Project

project = Project.access()
project_id = project.get_metadata()['metadata']['guid']
client.set.default_project(project_id)

'SUCCESS'

To proceed, you can go directly to the section [2. Save the SPSS model](#save).

<a id="wmls"></a>
### 1.2 Connecting to Watson Machine Learning Server <a id="wmls"></a>

In this subsection, you will learn how to set up the Watson Machine learning (WML) Server that is required to save, deploy, and score the model in the Watson Machine learning (WML) repository.

**Connect to the Watson Machine Learning Server using the Python client**<br><br>

<div class="alert-block alert-info">
    <br>
    &nbsp;&nbsp;&nbsp;&nbsp;To install the Watson Machine Learning Server, follow <a href="https://www.ibm.com/support/knowledgecenter/SS3PWM_1.0.0/wsj/wmls/wmls-install-over.html" target="_blank" rel="noopener no referrer">these documentation steps</a>.<br><br>
    &nbsp;&nbsp;&nbsp;&nbsp;To connect to the WML server and find your authentication information (your credentials) follow the steps provided here in the <a href="https://www.ibm.com/support/knowledgecenter/SS3PWM_1.0.0/wsj/wmls/wmls-connect.html" target="_blank" rel="noopener no referrer">Documentation</a>.
    <br><br>
</div>

**Action**: Enter your WML Server credentials in the following cell.

In [7]:
# Enter your credentials here.
wml_credentials = {
    'url': '<URL>:31843',
    'username': '---',
    'password': '---',
    'instance_id': 'wml_local',
    'version': '2.0'
}

In [8]:
# @hidden_cell

wml_credentials = {
    'url': 'https://wmlserver-dev-test.ml.test.cloud.ibm.com:31843',
    'username': 'admin',
    'password': 'password',
    'instance_id': 'wml_local',
    'version': '2.0'
}

Import the `watson-machine-learning-client` module and authenticate the service instance.

In [9]:
client = WatsonMachineLearningAPIClient(wml_credentials)

In [10]:
client.version

'1.0.112'

Since you are using WML Server in this section, you can obtain the space UID by using the following cells.

<div class="alert-block alert-info"><br>
&nbsp;&nbsp;&nbsp;&nbsp;You can create your own <a href="https://www.ibm.com/support/knowledgecenter/SS3PWM_1.0.0/wsj/analyze-data/ml-spaces_local.html" target="_blank" rel="noopener no referrer">deployment space</a> by selecting <b>Deployment Spaces</b> from the Navigation Menu on the top left of this page.<br><br></div>

Alternatively, you can create a deployment and obtain its UID using the code in the following cell.

In [11]:
# Obtain the UId of your space
def guid_from_space_name(client, space_name):
    instance_details = client.service_instance.get_details()
    space = client.spaces.get_details()
    return (next(item for item in space['resources']
                 if item['entity']['name'] == space_name)['metadata']['guid'])

**Action:** Enter the name of your deployment space in the code below: `space_uid = guid_from_space_name(client, 'YOUR_DEPLOYMENT_SPACE')`.

In [12]:
# Enter the name of your deployment space here:
space_uid = guid_from_space_name(client, 'YOUR_DEPLOYMENT_SPACE')
print('Space UID = {}'.format(space_uid))

Space UID = 3ce5e796-f127-431a-9609-46456e10d274


Setting the default space is mandatory when you use WML Server. You can set this using the cell below.

In [13]:
client.set.default_space(space_uid)

'SUCCESS'

<a id="save"></a>
## 2. Save the SPSS model

Download the SPSS sample model from the <a href="https://github.com/pmservice/wml-sample-models" target="_blank" rel="noopener no referrer">Git repository</a>.

<div class="alert-block alert-info"><br>
    &nbsp;&nbsp;&nbsp;&nbsp;<b>Note:</b> You may need to install the <tt>wget</tt> package. To install the <tt>wget</tt> package, run the following command. <br><br>
</div>

In [14]:
!pip install -q --upgrade wget

In [15]:
# Download the sample SPSS model.
from os import mkdir
from os.path import isdir, isfile, join
from wget import download

sample_dir = 'spss_sample_model'
if not isdir(sample_dir):
    mkdir(sample_dir)

filename = join(sample_dir, 'customer-satisfaction-prediction.str')
if not isfile(filename):
    filename = download(
        'https://github.com/pmservice/wml-sample-models/raw/master/spss/' +
        'customer-satisfaction-prediction/model/' +
        'customer-satisfaction-prediction.str', out=sample_dir)
print(filename)

spss_sample_model/customer-satisfaction-prediction.str


Save the downloaded SPSS sample model as *SPSS model for Churn prediction*. First, you need to create the model metadata.

In [16]:
# Save the SPSS model to the WML repository.
# Model Metadata.
software_spec_uid = client.software_specifications.get_uid_by_name(
    'spss-modeler_18.1')

meta_props = {
    client.repository.ModelMetaNames.NAME: 'SPSS model for Churn prediction',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
    client.repository.ModelMetaNames.TYPE: 'spss-modeler_18.1'
}

You can extract the model UID from the saved model details.

In [17]:
# Create the model artifact.
model_artifact = client.repository.store_model(filename, meta_props=meta_props)
model_uid = client.repository.get_model_uid(model_artifact)
print('Model UID = {}'.format(model_uid))

Model UID = ef8b3fbd-aa7e-413a-9cd4-c503db02e546


Obtain the saved model metadata using the model UID.

In [18]:
from json import dumps

# Details about the model.
model_details = client.repository.get_details(model_uid)
print(dumps(model_details, indent=4))

{
    "metadata": {
        "name": "SPSS model for Churn prediction",
        "guid": "ef8b3fbd-aa7e-413a-9cd4-c503db02e546",
        "id": "ef8b3fbd-aa7e-413a-9cd4-c503db02e546",
        "modified_at": "2020-08-28T16:51:05.002Z",
        "created_at": "2020-08-28T16:51:04.002Z",
        "owner": "1000330999",
        "href": "/v4/models/ef8b3fbd-aa7e-413a-9cd4-c503db02e546?space_id=3ce5e796-f127-431a-9609-46456e10d274",
        "space_id": "3ce5e796-f127-431a-9609-46456e10d274"
    },
    "entity": {
        "name": "SPSS model for Churn prediction",
        "content_status": {
            "state": "persisted"
        },
        "software_spec": {
            "id": "5c3cad7e-507f-4b2a-a9a3-ab53a21dee8b"
        },
        "space": {
            "id": "3ce5e796-f127-431a-9609-46456e10d274",
            "href": "/v4/spaces/3ce5e796-f127-431a-9609-46456e10d274"
        },
        "type": "spss-modeler_18.1"
    }
}


You can list all stored models using the `list_models` method.

In [19]:
# Display a list of all the models.
client.repository.list_models()

------------------------------------  -------------------------------  ------------------------  -----------------
GUID                                  NAME                             CREATED                   TYPE
ef8b3fbd-aa7e-413a-9cd4-c503db02e546  SPSS model for Churn prediction  2020-08-28T16:51:04.002Z  spss-modeler_18.1
------------------------------------  -------------------------------  ------------------------  -----------------


<div class="alert-block alert-info"><br>
&nbsp;&nbsp;&nbsp;&nbsp;From the list of stored models, you can see that model is successfully saved. <br>

&nbsp;&nbsp;&nbsp;&nbsp;With Watson Studio Desktop credentials, this means you have saved the model in your project.<br>&nbsp;&nbsp;&nbsp;&nbsp;You can see the saved model in your project UI by clicking on your project name in the breadcrumb at the top of the application. <br>

&nbsp;&nbsp;&nbsp;&nbsp;With WML Server credentials, this means that you have saved the model in your deployment space.<br>&nbsp;&nbsp;&nbsp;&nbsp;You can view your model by selecting <b>Deployment Spaces</b> from the Navigation Menu and clicking on your deployment space name.<br>
<br></div>

If you are using WML Server, proceed to section [3. Deploy the SPSS Model (WML Server only)](#deploy). If you are using Watson Studio Desktop, you may skip to the [summary](#summary).

## 3. Deploy the SPSS Model (WML Server only) <a id="deploy"></a>

### 3.1 Create a model deployment<a id="create"></a>

Using the code below to create an online deployment using the model UID obtained in the previous section. Create the deployment metadata using the code below.

In [20]:
# Deployment metadata.
deploy_meta = {
    client.deployments.ConfigurationMetaNames.NAME:
    'Sample SPSS model deployment',
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

In [21]:
# Create the deployment.
deployment_details = client.deployments.create(model_uid,
                                               meta_props=deploy_meta)



#######################################################################################

Synchronous deployment creation for uid: 'ef8b3fbd-aa7e-413a-9cd4-c503db02e546' started

#######################################################################################


initializing....
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='9d45ec75-80d0-4064-acd9-3307e4a4a7db'
------------------------------------------------------------------------------------------------




Obtain the list of all deployments in the deployment space.

In [22]:
# List the deployments.
client.deployments.list()

------------------------------------  ----------------------------  -----  ------------------------  -------------
GUID                                  NAME                          STATE  CREATED                   ARTIFACT_TYPE
9d45ec75-80d0-4064-acd9-3307e4a4a7db  Sample SPSS model deployment  ready  2020-08-28T16:51:08.301Z  model
------------------------------------  ----------------------------  -----  ------------------------  -------------


Now, you can check details of your deployments.

In [23]:
# Deployment UID.
deployment_uid = client.deployments.get_uid(deployment_details)
print('Deployment UID = {}'.format(deployment_uid))

Deployment UID = 9d45ec75-80d0-4064-acd9-3307e4a4a7db


In [24]:
# Deployment details.
print(dumps(client.deployments.get_details(deployment_uid), indent=4))

{
    "entity": {
        "asset": {
            "href": "/v4/models/ef8b3fbd-aa7e-413a-9cd4-c503db02e546?space_id=3ce5e796-f127-431a-9609-46456e10d274",
            "id": "ef8b3fbd-aa7e-413a-9cd4-c503db02e546"
        },
        "custom": {},
        "description": "",
        "name": "Sample SPSS model deployment",
        "online": {},
        "space": {
            "href": "/v4/spaces/3ce5e796-f127-431a-9609-46456e10d274",
            "id": "3ce5e796-f127-431a-9609-46456e10d274"
        },
        "space_id": "3ce5e796-f127-431a-9609-46456e10d274",
        "status": {
            "online_url": {
                "url": "https://wmlserver-dev-test.ml.test.cloud.ibm.com:31843/v4/deployments/9d45ec75-80d0-4064-acd9-3307e4a4a7db/predictions"
            },
            "state": "ready"
        }
    },
    "metadata": {
        "created_at": "2020-08-28T16:51:08.301Z",
        "description": "",
        "guid": "9d45ec75-80d0-4064-acd9-3307e4a4a7db",
        "href": "/v4/deployments/9d45

### 3.2 Score the model <a id="score"></a>

Prepare the scoring payload with values to score against the deployed model.

In [25]:
# Prepare scoring payload.
job_payload = {
    client.deployments.ScoringMetaNames.INPUT_DATA: [{
        'fields': [
            'customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
            'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
            'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
            'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
            'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges',
            'TotalCharges', 'Churn', 'SampleWeight'
        ],
        'values': [[
            '3638-WEABW', 'Female', 0, 'Yes', 'No', 58, 'Yes', 'Yes', 'DSL',
            'No', 'Yes', 'No', 'Yes', 'No', 'No', 'Two year', 'Yes',
            'Credit card (automatic)', 59.9, 3505.1, 'No', 2.768
        ]]
    }]
}
print(dumps(job_payload, indent=4))

{
    "input_data": [
        {
            "fields": [
                "customerID",
                "gender",
                "SeniorCitizen",
                "Partner",
                "Dependents",
                "tenure",
                "PhoneService",
                "MultipleLines",
                "InternetService",
                "OnlineSecurity",
                "OnlineBackup",
                "DeviceProtection",
                "TechSupport",
                "StreamingTV",
                "StreamingMovies",
                "Contract",
                "PaperlessBilling",
                "PaymentMethod",
                "MonthlyCharges",
                "TotalCharges",
                "Churn",
                "SampleWeight"
            ],
            "values": [
                [
                    "3638-WEABW",
                    "Female",
                    0,
                    "Yes",
                    "No",
                    58,
                    "Yes",
      

Use the client's `score` method to score test data.

In [26]:
# Perform prediction and display the result.
job_details = client.deployments.score(deployment_uid, job_payload)
print(dumps(job_details, indent=4))

{
    "predictions": [
        {
            "fields": [
                "customerID",
                "Churn",
                "Predicted Churn",
                "Probability of Churn"
            ],
            "values": [
                [
                    "3638-WEABW",
                    "No",
                    "No",
                    0.0526309571556145
                ]
            ]
        }
    ]
}


As you can see, this sample telco customer is satisfied ("Predicted Churn", "No").

### 3.3 Delete the deployment and model<a id='delete'></a>

Use the following method to delete the deployment.

In [27]:
client.deployments.delete(deployment_uid)

'SUCCESS'

In [28]:
client.deployments.list()

----  ----  -----  -------  -------------
GUID  NAME  STATE  CREATED  ARTIFACT_TYPE
----  ----  -----  -------  -------------


You can delete the model as well by running the following cell.

In [29]:
client.repository.delete(model_uid)

'SUCCESS'

You can check that your model was deleted by generating a list of your saved models.

In [30]:
client.repository.list_models()

----  ----  -------  ----
GUID  NAME  CREATED  TYPE
----  ----  -------  ----


<a id="summary"></a>
## 4. Summary     

You successfully completed this notebook! 

You learned how to save an SPSS model both on Watson Studio Desktop and Watson Machine Learning Server. You can now deploy and score the SPSS model on WML Server as well.

### Resources <a id="resources"></a>

To learn more about configurations used in this notebook or more sample notebooks, tutorials, documentation, how-tos, and blog posts, check out these links:

<div class="alert alert-block alert-success"><a id="resources"></a>

<h4>IBM documentation</h4>
<br>
 <li> <a href="https://wml-api-pyclient-dev-v4.mybluemix.net" target="_blank" rel="noopener no referrer">watson-machine-learning</a></li> 
 <li> <a href="https://www.ibm.com/support/knowledgecenter/SS3PWM_1.0.0/wsj/wmls/overview.html" target="_blank" rel="noopener noreferrer">Watson Machine Learning Server</a></li>
 
<h4> IBM Samples</h4>
<br>
 <li> <a href="https://github.com/IBMDataScience/sample-notebooks" target="_blank" rel="noopener noreferrer">Sample notebooks</a></li>
 
<h4> Others</h4>
<br>
 <li> <a href="https://www.python.org" target="_blank" rel="noopener noreferrer">Official Python website</a></li>
 </div>

### Authors

**Lukasz Cmielowski**, Ph.D., is an Automation Architect and Data Scientist at IBM with a track record of developing enterprise-level applications that substantially increase the clients' ability to turn data into actionable knowledge.  
**Jihyoung Kim**, Ph.D., is a Data Scientist at IBM who contributes to Watson Studio to democratize data science.

Copyright © 2017-2020 IBM. This notebook and its source code are released under the terms of the MIT License.

<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>